# 線形MDPにおける一様PACなアルゴリズム

参考：
* [Uniform-PAC Bounds for Reinforcement Learning with Linear Function Approximation](https://arxiv.org/abs/2106.11612)

今回は線形MDPで[一様PAC](RL_uniform_PAC.ipynb)を達成するアルゴリズムについて学びます．
先に線形バンディットで考えてみましょう．

## 線形バンディット

* $\mathcal{D}$を行動の集合とします（論文では$k$に依存させてますが，面倒なので割愛します）．
* 各ラウンド$k$でエージェントは行動$x_k \in \mathcal{D}$を選択することを考えます．
* 報酬は$r_k=\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle+\epsilon_k$, where $\boldsymbol{\mu}^* \in \mathbb{R}^d$の形式．
    * 簡単のため，$\left\|\boldsymbol{\mu}^*\right\|_2 \leq 1$とします．
    * ここで$\epsilon_k$は次を満たすsub-gaussianなノイズです．
        * $\forall k \in \mathbb{N}, \lambda \in \mathbb{R}, \mathbb{E}\left[e^{\lambda \epsilon_k} \mid \mathbf{x}_{1: k}, \epsilon_{1: k-1}\right] \leq \exp \left(\lambda^2 / 2\right),\left\|\mathbf{x}_k\right\|_2 \leq 1,\left\|\boldsymbol{\mu}^*\right\|_2 \leq 1$
* このとき，次を満たす場合にアルゴリズムは$(\varepsilon, \delta)$-一様PACと呼びます（ここで$\Delta_k$は$k$ラウンド目でのsuboptimalityです）
$$
\mathbb{P}\left(\exists \epsilon>0, \sum_{k=1}^{\infty} \mathbb{1}\left\{\Delta_k:=\max _{\mathbf{x} \in \mathcal{D}_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle>\epsilon\right\}>f(\epsilon, \delta)\right)<\delta,
$$

### 一様PACなアルゴリズム

[RL_uniform_PAC](RL_uniform_PAC.ipynb)でやりましたが，普通にPotential lemmaでConfidence ballのバウンドをしても，その平均が小さくなることは言えますが，一様PACにはなりません．そこで，次のアルゴリズムを考えましょう：

準備：
* $k$ラウンドまでの過去のデータ$1, 2, 3, \dots, k$を，重なりのない集合$C^1, C^2, \dots, C^{S_k}$に分解することを考えます．ここで，$S_k$はその集合の総数です．

* for round $k=1, 2, \dots, $
    * for all level $l=1, 2, \dots, S_k$でグラム行列の計算と，パラメータの推定をします：
        * $\boldsymbol{\Sigma}_k^l=\lambda \mathbf{I}+\sum_{i \in \mathcal{C}^l} \mathbf{x}_i \mathbf{x}_i^{\top}$
        * $\mathbf{\mu}_k^l=\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \sum_{i \in \mathcal{C}^l} \mathbf{x}_i r_i$
    * OFUに従って行動を選択します（ここで，各$C^l$についてもOFUが成立するので，Optimisticかつ，ボーナスが最も小さくなるようなUncertainty集合を利用していることに注意しましょう）：
    $\mathbf{x}_k \leftarrow \operatorname{argmax}_{\mathbf{x} \in \mathcal{D}} \min _{1 \leq l \leq S_k}\left(\mathbf{\mu}_k^l\right)^{\top} \mathbf{x}+\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}}$
    * 今回得られたデータ$x_k, r_k$を，適切なデータセット$C^{l_k}$に割り当てます．詳細は後述．
    * $S_{k+1}$を$\max _{l:\left|\mathcal{C}^l\right|>0} l$と設定します．（つまり要素が入っている$\mathcal{C}$の個数）


**データの$\mathcal{C}$への割当方法**

$l_k$は次の手順で割り当てます：

* $l_k = 1$とします．
* while $\sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k}\right)^{-1} \mathbf{x}_k} \leq 2^{-l_k}$かつ$l_k \leq S_k$の間，
    * $l_k \leftarrow l_k +1 $と増やします
* $k$ラウンドでのデータを$\mathcal{C}^{l_k}$に追加します．

これはつまり，各$C^l$における$x_k$の不確実性を全て評価して，一定以上の不確実性があり，かつ最もその値が小さい集合に$C^l$を突っ込んでいます．

![uPAC-linearMDP](figs/uniform-linearMDP.jpg)


このしきい値はPeelingテクニックがベースのアイデアになってます．
UniformPACで重要なのは，$k \to \infty$に対してUnion boundを取ることです（[RL_uniform_PAC](RL_uniform_PAC.ipynb)でもやりましたね）
上の分割方法を考えると，各$C^l$のサイズを小さく抑えることができ，各$C^l$についてのUnion boundで全体をバウンドすることができます．
なので実質やってることはPeelingテクニックと同じです．

### リグレット解析

まず，いつものようにYadkoriのElliptical potential lemmaを使ってボーナスの総和をバウンドします（$\lambda=1$としておきます）：

---

任意のベクトル$\left\{\mathbf{x}_k\right\}_{k=1}^K$ in $\mathbb{R}^d$について，
$$
\boldsymbol{\Sigma}_k=I+\sum_{i=1}^k \mathbf{x}_i \mathbf{x}_i^{\top}
$$
とします．ここで，$\left\|\mathbf{x}_k\right\|_2 \leq L$ならば，
$$
\sum_{k=1}^K\left\|\mathbf{x}_k\right\|_{\boldsymbol{\Sigma}_{k-1}^{-1}}^2 \leq 2 d \log \frac{d +K L^2}{d}
$$
が成り立ちます．

---

これを使うと，各$C^l$のサイズがバウンドできます．

---

#### $C^l$のサイズのバウンド

適当なラウンド$k$と適当な$l$を考えましょう．また，その要素数を
$\mathcal{C}_k^l=\left\{k_1, . ., k_m\right\}$とします．ここで$1 \leq k_1<k_2<. .<k_m<k$です．
* **つまり，$\mathcal{C}_k^l$にはラウンド$k_1, k_2, \dots k_m$で要素が追加された状況です．**

アルゴリズムの$C^l$の追加方法を思い出しましょう：
* while $\sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k}\right)^{-1} \mathbf{x}_k} \leq 2^{-l_k}$かつ$l_k \leq S_k$の間，
    * $l_k \leftarrow l_k +1 $と増やします
* $k$ラウンドでのデータを$\mathcal{C}^{l_k}$に追加します．

すなわち，次の２つが言えます：
1. 最初の追加タイミング$k_1$は$C^l$を作るので，$k_1$では$S_{k_1} < l$です．しかし，それ以降の$k_2, k_3, k_m$では既存の$C^l$に追加しているので，while文の条件から，$l \leq S_{k_i}$が成立します
2. よって，$k_2, k_3, k_m$では，while文のもう一つの条件によって要素の追加が行われています．すなわち，$\mathbf{x}_{k_i}^{\top}\left(\boldsymbol{\Sigma}_{k_i}^l\right)^{-1} \mathbf{x}_{k_i} \geq 4^{-l}$も成り立ちます．
ここで$\boldsymbol{\Sigma}_{k_i}^l=\lambda \mathbf{I}+\sum_{j=1}^{i-1} \mathbf{x}_{k_j} \mathbf{x}_{k_j}^{\top}$です．

よって，$2\leq i \leq m$で和を取ると，
$$
\sum_{i=1}^m \mathbf{x}_{k_i}^{\top}\left(\boldsymbol{\Sigma}_{k_i}^l\right)^{-1} \mathbf{x}_{k_i} \geq \sum_{i=2}^m \mathbf{x}_{k_i}^{\top}\left(\boldsymbol{\Sigma}_{k_i}^l\right)^{-1} \mathbf{x}_{k_i} \geq(m-1) 4^{-l}
$$
が言えます．一方で，Elliptical Potential Lemmaから，
$$
\sum_{i=1}^m \mathbf{x}_{k_i}^{\top}\left(\boldsymbol{\Sigma}_{k_i}^l\right)^{-1} \mathbf{x}_{k_i} \leq 2 d \log (1+m / d)
$$
が言えます．よって，
$$
(m-1) 4^{-l} \leq 2 d \log (1+m / d)
$$
です．つまり，各$C^l_k$のサイズは
$$
\left|\mathcal{C}_k^l\right|=m \leq 17 d l 4^l
$$
でバウンドされます．

---

#### 信頼区間の半径のバウンド

続いて，信頼区間の中に真のパラメータが入ることも証明しておきます．基本的にはYadkoriの応用です．

---

$\beta_l = 6 \sqrt{d l \log (d l / \delta)}$とします．任意の$l, k$について，確率$1-\delta$以上で，

$$
\left\|\mathbf{\mu}_k^l-\boldsymbol{\mu}^*\right\|_{\boldsymbol{\Sigma}_k^l} \leq \beta_l
$$

が成り立ちます．

**証明**

次を利用します．自然な表記と設定のもとで（詳細は論文に任せる），

* $\left\langle\mathbf{x}_t, \boldsymbol{\mu}^*\right\rangle+\epsilon_t$
* $\boldsymbol{\Sigma}_t=I+\sum_{i=1}^t \mathbf{x}_i \mathbf{x}_i^{\top}, \mathbf{b}_t=\sum_{i=1}^t \mathbf{x}_i y_i, \mathbf{\mu}_t=\left(\boldsymbol{\Sigma}_t\right)^{-1} \mathbf{b}_t$

について，もし$\left\|\boldsymbol{\mu}^*\right\| \leq 1$ and $\left\|\mathbf{x}_t\right\| \leq 1$ならば，
$$
\left\|\boldsymbol{\mu}^*-\mathbf{\mu}_t\right\|_{\boldsymbol{\Sigma}_t} \leq R \sqrt{d \log \left(\frac{1+t}{\delta}\right)}+1
$$
が成り立ちます．

これを踏まえて，本来の証明に戻りましょう．固定された$l$について考えます．

* $k_0=0$
* 各$k_1, k_2, \dots$について，$k_i=\min \left\{k: k>k_{i-1}, l_k=l\right\}$を，行動が$C^l$に追加されたラウンドとします．

ここで，改めてアームの引き方と報酬の発生について思い出しましょう：
* $x_k$を観測 → $C^{l_k}$に$k$を追加 → $r_k$を観測

よって，任意の$k_i<k \leq k_{i+1}$では，$C^l$に結果が追加されてないので，

$$
\boldsymbol{\Sigma}_k^l=\mathbf{I}+\sum_{j=1}^i \mathbf{x}_{k_j} \mathbf{x}_{k_j}^{\top}, \mathbf{b}_k^l=\sum_{j=1}^i \mathbf{x}_{k_j} r_{k_j}, \mathbf{\mu}_k^l=\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{b}_k^l
$$

が成り立ちます．
適切なFiltrationを考えると，上のYadkoriの定理から，任意の$i \geq 0$について，確率$\delta / (l (l+1))$以上で

$$
\left\|\boldsymbol{\mu}^*-\mathbf{\mu}_{k_{i+1}}^l\right\|_{\boldsymbol{\Sigma}_{k_{i+1}}^l} \leq \sqrt{d \log \left(\frac{i+1}{\delta /(l(l+1))}\right)}+1 .
$$

が成り立ちます．ここで，
* 分子の$i+1$は$\mu$を回帰するために使ったデータの数
* $1/l(l+1)$は後で任意の$l$についてUnion boundを取るときに，
$
\sum_{l=1}^{\infty}(\delta /(l(l+1)))=\delta,
$
を利用したいからやってるっぽい．

これまでの結果を合体させると，任意の$k_i<k \leq k_{i+1}$について，
$$
\left\|\boldsymbol{\mu}^*-\mathbf{\mu}_k^l\right\|_{\boldsymbol{\Sigma}_k^l} \leq \sqrt{d \log \left(\frac{i+1}{\delta /(l(l+1))}\right)}+1
$$

が成り立ちます．また，$\left|\mathcal{C}^l\right| \leq 17 d l 4^l$であることを踏まえると，$i \leq 17 d l 4^l$でUnion boundを取れば，任意の$k \in \mathbb{N}$についての議論ができます．すなわち，

$$
\left\|\boldsymbol{\mu}^*-\mathbf{\mu}_k^l\right\|_{\boldsymbol{\Sigma}_k^l} \leq \sqrt{d \log \left(\frac{17 d l 4^l+1}{\delta /(l(l+1))}\right)}+1 \leq \beta_l \quad \forall k \in \mathbb{N}
$$
が成立します．最後に$1 - \sum_{l=1}^{\infty}(\delta /(l(l+1)))=1-\delta$のUnion boundを取れば，
$\left\|\mathbf{\mu}_k^l-\boldsymbol{\mu}^*\right\|_{\boldsymbol{\Sigma}_k^l} \leq \beta_l$
が任意の$l$で成立します．


---

#### リグレットのバウンド

最後に，今までのを全て合体してリグレットを導出します．

半径のバウンドを起こすイベントが成立しているとしましょう．このとき，任意のレベル$l \in \mathbb{N}$, ラウンド $k \in \mathbb{N}$ そして行動 $\mathbf{x} \in \mathcal{D}$について，

$$
\begin{aligned}
&\left(\mathbf{\mu}_k^l\right)^{\top} \mathbf{x}+\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}}-\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle \\
= &\left(\mathbf{\mu}_k^l-\boldsymbol{\mu}^*\right)^{\top} \mathbf{x}+\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}} \\
\geq & \beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}}-\left\|\mathbf{\mu}_k^l-\boldsymbol{\mu}^*\right\|_{\boldsymbol{\Sigma}_k^l}\|\mathbf{x}\|_{\left(\boldsymbol{\Sigma}_k^l\right)^{-1}} \\
\geq &\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1}}-\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1}} \\
=&0
\end{aligned}
$$

が成立するので，
$$
\min _{1 \leq l \leq S_k}\left(\mathbf{\mu}_k^l\right)^{\top} \mathbf{x}+\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}} \geq \min _{1 \leq l \leq S_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle=\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle
$$
であり，アルゴリズムのアームの選び方にOptimismが成り立ってます．
よって，即時リグレットは
$$
\begin{aligned}
&\max _{\mathbf{x} \in \mathcal{D}_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle \\
& \leq \max _{\mathbf{x} \in \mathcal{D}_k} \min _{1 \leq l \leq S_k}\left(\mathbf{\mu}_k^l\right)^{\top} \mathbf{x}+\beta_l \sqrt{\mathbf{x}^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}}-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle \\
& =\min _{1 \leq l \leq S_l}\left(\mathbf{\mu}_k^l\right)^{\top} \mathbf{x}_k+\beta_l \sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^l\right)^{-1} \mathbf{x}_k}-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle,
\end{aligned}
$$

でバウンドされます．
また，「ボーナスが$2^{-\left(l_k\right)}$をぎりぎり超える」ように$l_k$が設定されているので，$l_k - 1$は$\sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1} \mathbf{x}_k} \leq 2^{-\left(l_k-1\right)}$を満たすことから，

$$
\begin{aligned}
&\max _{\mathbf{x} \in \mathcal{D}_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle \\
& \leq\left(\mathbf{\mu}_k^{l_k-1}\right)^{\top} \mathbf{x}_k+\beta_{l_k-1} \sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1} \mathbf{x}_k}-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle \\
& =\left(\mathbf{\mu}_k^{l_k-1}-\boldsymbol{\mu}^*\right)^{\top} \mathbf{x}_k+\beta_{l_k-1} \sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1} \mathbf{x}_k} \\
& \leq\left\|\mathbf{\mu}_k^{l_k-1}-\boldsymbol{\mu}^*\right\|_{\boldsymbol{\Sigma}_k^{l_k-1}}\left\|\mathbf{x}_k\right\|_{\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1}}+\beta_{l_k-1} \sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1} \mathbf{x}_k} \\
& \leq 2 \beta_{l_k-1} \sqrt{\mathbf{x}_k^{\top}\left(\boldsymbol{\Sigma}_k^{l_k-1}\right)^{-1} \mathbf{x}_k} \\
& \leq 2 \beta_{l_k-1} \times 2^{-\left(l_k-1\right)},
\end{aligned}
$$
が成り立ちます．

さて，$\beta_l=6 \sqrt{d l \log (d l / \delta)}$なので，適切な$C$について
$$2^l \geq C \sqrt{d \log ^2(d /(\delta \epsilon)) / \epsilon}$$
を満たすような十分大きい$l$ならば，$2 \beta_{l-1} \times 2^{-(l-1)} \leq \epsilon$が言えます．

このような最小のレベルを$m=\left[\log \left(C \sqrt{d \log ^2(d /(\delta \epsilon))} / \epsilon\right)\right]$と置きます．
つまり，任意の$l_k > m$であるような$k$については，
$$
\max _{\mathbf{x} \in \mathcal{D}_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle \leq 2 \beta_{l_k-1} \times 2^{-\left(l_k-1\right)} \leq \epsilon .
$$
が成り立ちます．よって，任意の$\epsilon > 0$について，
$$
\begin{aligned}
&\sum_{k=1}^{\infty} \mathbb{1}\left\{\max _{\mathbf{x} \in \mathcal{D}_k}\left\langle\boldsymbol{\mu}^*, \mathbf{x}\right\rangle-\left\langle\boldsymbol{\mu}^*, \mathbf{x}_k\right\rangle>\epsilon\right\}  \quad （\epsilon 非最適アームを選ぶ回数）\\
\leq &\sum_{k=1}^{\infty} \mathbb{1}\left\{l_k \leq m\right\} \quad （l_kがm以下である回数）\\
=& \sum_{k=1}^{\infty} \sum_{l=1}^m \mathbb{1}\left\{l_k=l\right\} \\
=&\sum_{l=1}^m \underbrace{\sum_{k=1}^{\infty} \mathbb{1}\left\{l_k=l\right\}}_{全ラウンドでlが選択される総回数 \leq 17 d l 4^l}\\
\leq & C^{\prime} d^2 \log ^3(d /(\delta \epsilon)) / \epsilon^2
\end{aligned}
$$

が成り立ちます．ここで，最後の不等式は$\mathrm{t} \sum_{l=1}^m l 4^l \leq m 4^{m+1}$を使いました．
